<a href="https://colab.research.google.com/github/Five-Star-Station/Hansol_QA/blob/main/simso/test_question_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845034 sha256=a00350ee9271b972bcafbe84c16cefd9e642bef746b75c51bcced072170bf76b
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model


In [2]:
import re

import numpy as np
import pandas as pd

from kiwipiepy import Kiwi

In [3]:
base_path = '/content/drive/MyDrive/Colab Notebooks/data/dacon-hansol-qa/'
train_df = pd.read_csv(base_path + 'train.csv')
test_df = pd.read_csv(base_path + 'test.csv')
submission_df = pd.read_csv(base_path + 'sample_submission.csv')

In [4]:
kiwi = Kiwi()

In [26]:
test_queries = [
    "다이닝 룸을 더 고급스럽게 꾸밀려면 어떻게 해야 하나요와 아레카 야자를 키우면 어떤 이점이 있는지의 관계는 무엇인가요?",
    "물 누수로 인한 곰팡이와 곰팡이 냄새를 제거하는 데 사용할 수 있는 가정용 청소용품이 있을까요? 그리고 펫테리어가 무엇인지 자세히 알려주세요.",
    "입구나 복도의 표면에 사용하기 적합한 페인트 종류는 무엇이며, 이 공간을 환영스럽게 꾸미는 데 있어 어떤 인테리어 요소가 중요한가요?",
    "아파트 도배 평수를 계산하는 방법과 소화기 설치 시 주의해야 할 사항에 대해 설명해줄 수 있나요?",
    "도배지가 찢어진 경우 터짐 하자가 발생하는 원인과 그에 따른 책임소재는 무엇인가요? 그리고 이를 해결하기 위해 어떤 방법을 사용할 수 있나요?",
    "어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?"
]

In [27]:
def preprocess_query(query: str) -> list[str]:
    # 1. 문장 나누기
    pattern = r"(.+)와 (.+)의 관계는 무엇인가요\?"
    matches = re.findall(pattern, query)
    if matches:
        queries = [*matches[0]]
    else:
        queries = [q.text for q in kiwi.split_into_sents(query)]

    # 2. 문장 다듬기
    # 삭제할 형태소
    stoptags = ['SSC', 'SSO', 'SF']
    # MAG 형태소 중 삭제할 것만
    stopwords = ['좀', '자세히', '더', '빨리', '특히', '없이', '너무', '가장', '많이', '특별히', '바로', '주로', '또한', '그리고', ',']
    processed_queries = []
    for q in queries:
        tokens = kiwi.tokenize(q)
        selected_tokens = []
        for t in tokens:
            if t.form == '설명' or t.tagged_form == '알리/VV':
                break
            if t.tag not in stoptags and t.form not in stopwords:
                selected_tokens.append(t)
            else:
                continue
        if len(selected_tokens) > 0:
            q_new = kiwi.join(selected_tokens)
        else:
            continue

        # 나머지 처리...
        if len(processed_queries) > 0 and q_new.startswith('이를 '):
            q_new_prev = processed_queries.pop()
            processed_queries.append(q_new_prev + " " + q_new)
        elif q_new.find('방법과 ') >= 0:
            q_new_split = q_new.split('과 ')
            processed_queries += q_new_split
        else:
            processed_queries.append(q_new)
    # 쿼리 사용 시에 split(sep='/')으로 나누기
    return "/".join(processed_queries)

In [28]:
list(map(lambda x:preprocess_query(x), test_queries))

['다이닝 룸을 고급스럽게 꾸밀려면 어떻게 해야 하나요/아레카 야자를 키우면 어떤 이점이 있는지',
 '물 누수로 인한 곰팡이와 곰팡이 냄새를 제거하는 데 사용할 수 있는 가정용 청소용품이 있을까요/펫테리어가 무엇인지',
 '입구나 복도의 표면에 사용하기 적합한 페인트 종류는 무엇이며 이 공간을 환영스럽게 꾸미는 데 있어 어떤 인테리어 요소가 중요한가요',
 '아파트 도배 평수를 계산하는 방법/소화기 설치 시 주의해야 할 사항에 대해',
 '도배지가 찢어진 경우 터짐 하자가 발생하는 원인과 그에 따른 책임소재는 무엇인가요 이를 해결하기 위해 어떤 방법을 사용할 수 있나요',
 '어떤 상황에 개별 공간이 적합한지 어떤 상황에 오픈 플랜 공간이 적합한지/합지벽지의 어떤 단점이 있나요']

In [29]:
test_df['질문'].map(lambda x: preprocess_query(x))

0      방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요/원목사이딩을 사용하는 것에...
1                 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요
2                    큐블럭의 단점을/압출법 단열판을 사용하는 것의 장점은 무엇인가요
3       철골구조를 사용하는 고층 건물에서 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요
4                        도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요
                             ...                        
125    분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요/아파트 도배 평수를 어떻게 ...
126                                 압출법 보온판의 큰 장점은 무엇인가요
127    평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128                석고수정이 발생하는 큰 원인은 무엇인가요 이를 해결하는 방법에 대해
129             카페트의 기대 수명은 얼마나 될까요/오리지널징크의 장점에는 무엇이 있나요
Name: 질문, Length: 130, dtype: object

In [30]:
test_df['question_split'] = test_df['질문'].map(lambda x: preprocess_query(x))

In [ ]:
test_df.to_csv('/content/test_split.csv', index=False)